# Hiperparâmetro x Parâmetro

>

# Otimização de Hiperparâmetros

Problema? 

# Grid Search

Talvez a primeira coisa que qualquer pessoa pense ao se deparar com esse problema é criar listas de parâmetros e testar todas as combinações possíveis. Logo, ao tentar otimizar 2 hiperparâmetros com três possibilidades em cada, testaremos então $3 \times 3 = 9$ modelos. O modelo que tiver melhor desempenho, será o escolhido.

<img src="https://s3-sa-east-1.amazonaws.com/lcpi/14f61637-c80b-4ec3-8a1f-0c6e547ecb8d.png" style="height: 250px"/>

## Plano

1. ler dataset titanic
2. Criar features (X) e target (y)
3. Dividir dataset em treino e teste
4. Aplicar tecnicas do começo da aula para missing values e categorias
5. Treinar um modelo. **Qual?**
6. avaliar o modelo: **qual métrica?**

e finalmente...

7. Fazer o grid search
8. Discussão!

In [1]:
# passo 1
import pandas as pd
import numpy as np 

df = pd.read_csv('titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# passo 2
X = df[['Sex', 'Age', 'Fare']].copy()
y = df['Survived'].copy()

In [4]:
# passo 3
from sklearn.model_selection import train_test_split

X_train_tmp, X_test, y_train_tmp, y_test = train_test_split(X, y, test_size=0.3, random_state=13)

print(X_train_tmp.shape, X_test.shape, y_train_tmp.shape, y_test.shape)

(623, 3) (268, 3) (623,) (268,)


In [5]:
# create validation set
X_train, X_val, y_train, y_val = train_test_split(X_train_tmp, y_train_tmp, test_size=0.3, random_state=13)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(436, 3) (187, 3) (436,) (187,)


In [6]:
# passo 4

X_train['Sex'].value_counts()

male      287
female    149
Name: Sex, dtype: int64

In [7]:
# passo 4
from sklearn.preprocessing import OrdinalEncoder

ordinal = OrdinalEncoder()

X_train = X_train.copy()
X_val = X_val.copy()
X_test = X_test.copy()

ordinal.fit(X_train[['Sex']])

X_train['Sex'] = ordinal.transform(X_train[['Sex']])
X_val['Sex'] = ordinal.transform(X_val[['Sex']])
X_test['Sex'] = ordinal.transform(X_test[['Sex']])

# warnings são incompatibilidade entre pandas e sklearn!

In [8]:
X_train['Sex'].value_counts()

1.0    287
0.0    149
Name: Sex, dtype: int64

In [9]:
# passo 4 continuação
X_train.isnull().sum()

Sex      0
Age     95
Fare     0
dtype: int64

In [10]:
# passo 4 continuação
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')

imputer.fit(X_train[['Age']])

X_train['Age'] = imputer.transform(X_train[['Age']])
X_val['Age'] = imputer.transform(X_val[['Age']])
X_test['Age'] = imputer.transform(X_test[['Age']])

# warnings são incompatibilidade entre pandas e sklearn!

In [18]:
# passo 5
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

model.fit(X_train, y_train)

RandomForestClassifier()

In [19]:
# passo 6: qual métrica?
y.value_counts(dropna=False, normalize=True)

0    0.616162
1    0.383838
Name: Survived, dtype: float64

In [20]:
# passo 6
from sklearn.metrics import roc_auc_score

y_pred = model.predict_proba(X_val)

roc_auc_score(y_val, y_pred[:,1])

0.8295896066051481

In [21]:
model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

## Otimizar os hiperparâmetros

In [38]:
# passo 7: encontrando os melhores hiperparâmetros
from sklearn.model_selection import GridSearchCV

modelo = KNeighborsClassifier()

grid_search = {
    'n_neighbors': np.arange(5,20,1), #np.random.randint(5,100, 50), #[5, 50, 100],
    'p': [1,2,3]
}

gscv = GridSearchCV(estimator= modelo,
                    param_grid= grid_search, 
                    scoring='roc_auc' ,
                    cv=3)

gscv.fit(X_train, y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'n_neighbors': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]),
                         'p': [1, 2, 3]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='roc_auc', verbose=0)

In [39]:
# Quais os melhores parâmetros?
gscv.best_params_

{'n_neighbors': 12, 'p': 1}

In [40]:
gscv.best_score_

0.7352253209863199

In [41]:
# Qual a performance? Queremos otimizar o score de dados que não foram vistos!
y_pred = gscv.predict_proba(X_val)

roc_auc_score(y_val, y_pred[:,1])

0.7354905293831958

## Exercicio

Vamos testar e otimizar outro modelo?

Crie um modelo de árvore para os dados que já temos no problema do titanic, meça sua performance e em seguida utilize o grid search para otimizar 3 hiperparâmetros de sua escolha. 

**Para pensar:** Quais hiperparâmetros podem ser utlizados?

In [50]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(random_state=13)

tree.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=13, splitter='best')

In [51]:
y_pred = tree.predict_proba(X_val)
roc_auc_score(y_val, y_pred[:,1])

0.7557066537153958

In [62]:
tree.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': 'deprecated',
 'random_state': 13,
 'splitter': 'best'}

In [58]:
modelo = DecisionTreeClassifier(random_state=13)

grid_search = {
    'max_depth': np.arange(5,20,1),
    'criterion': ['gini', 'entropy'],
    'min_samples_split': np.arange(2, 10, 1)
}

gscv = GridSearchCV(estimator= modelo,
                    param_grid= grid_search, 
                    scoring='roc_auc' ,
                    cv=3)

gscv.fit(X_train, y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=13,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'criterion': ['gini', 'entropy'],
                     

In [59]:
y_pred = gscv.predict_proba(X_val)
roc_auc_score(y_val, y_pred[:,1])

0.8311680427391938

In [60]:
gscv.best_params_

{'criterion': 'entropy', 'max_depth': 5, 'min_samples_split': 7}

# Random Search

Funciona de forma muito similar ao Grid Search, criando um grid de parâmetros a serem testados. Porém, com a diferença de que a seleção dos valores escolhidos para teste é *aleatória* dado um conjunto de possibilidades. Ao invés de testar todos com todos, essa metodologia escolhe um valor ao acaso a partir de um conjunto pré-determinado. Mas uma escolha aleatória trás ganho?

Sim! [Alguns pesquisadores](https://s3-sa-east-1.amazonaws.com/lcpi/879e38fe-51f5-4f0e-abf1-339de202611b.pdf) perceberam que o Grid Search trás um problema na otimização dos valores: vamos supor que estamos testando os hiperparâmetros W, X, Y e Z, sendo $w$, $x$, $y$ e $z$ a quantidade de valores que cada hiperparâmetro receberá. Porém, modificar o W, por exemplo, trará uma variação desprezível no resultado do nosso modelo. Mas mesmo assim, por estar especificada no Grid Search, essa metodologia irá testar $ w \times x \times y \times z$, o que pode ser um desperdício. 

Além disso, ao especificar pontos estamos colocando um viés e pode ser que acabemos por não explorar espaços com máximos locais.

<img src="https://s3-sa-east-1.amazonaws.com/lcpi/ccb2c843-3e26-4f56-bca5-e380d6964751.png" style="height: 250px"/>


In [89]:
#implementar random search
from sklearn.model_selection import RandomizedSearchCV

modelo = DecisionTreeClassifier(random_state=13)

grid_search = {
    'max_depth': np.random.randint(1,20, 10),
    'criterion': ['gini', 'entropy'],
    'min_samples_split': np.random.randint(1,20, 10)
}

rscv = RandomizedSearchCV(estimator= modelo,
                          param_distributions= grid_search, 
                          scoring='roc_auc' ,
                          cv=3)

rscv.fit(X_train, y_train)

RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=13,
            

In [90]:
# Quais os melhores parâmetros?
rscv.best_params_

{'min_samples_split': 5, 'max_depth': 4, 'criterion': 'entropy'}

In [91]:
# Qual a performance?
y_pred = rscv.predict_proba(X_val)
roc_auc_score(y_val, y_pred[:,1])

0.8332928606119474

In [92]:
rscv.cv_results_ # note que encontramos um score muito com com apenas 10 tentativas

{'mean_fit_time': array([0.00336464, 0.00227944, 0.00225957, 0.00356968, 0.00243227,
        0.00241653, 0.00208521, 0.00218773, 0.00259741, 0.00216047]),
 'std_fit_time': array([0.00040006, 0.00040729, 0.00040416, 0.00176587, 0.00013909,
        0.00023346, 0.00013388, 0.00036023, 0.00075893, 0.00046842]),
 'mean_score_time': array([0.00291959, 0.00179601, 0.00229136, 0.00265376, 0.00271614,
        0.00248973, 0.00182756, 0.00310151, 0.00188239, 0.00181746]),
 'std_score_time': array([0.00075175, 0.00018619, 0.00052695, 0.00044946, 0.00016007,
        0.0005513 , 0.0002668 , 0.00090997, 0.00052654, 0.00021167]),
 'param_min_samples_split': masked_array(data=[17, 17, 8, 8, 5, 10, 12, 17, 10, 8],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'param_max_depth': masked_array(data=[16, 5, 4, 8, 4, 5, 18, 8, 10, 5],
              mask=[False, False, False, False, False, Fal

## Exercicio

Otimize seu modelo de knn usando o random search.